# Multidimensional Scaling

This is another way to do dimensionality reduction.  
Designed to bring objects, aka stimuli, down to a low-dimensional (Euclidean) space.
Basically, reduce things to 2 or 3 dimensions, and ensure distances represent difference between stimuli.

So, factor analysis had interpretations of factors as high-dimensional vectors, which we project down to a lower-dimensional space.
MDS is thought of in terms of points, brought down to low-dimensional space.

## Measuring Similarity

We have multiple approaches to measuring similarity between two stimuli.  
In other analysis approaches, the similarity measures we used were correlations and covariances.
However, there's different ways we can do this (we can also equivalently have measures of *dissimilarity*)
For MDS, we have two categories:

1. Direct Similarities
    - Pairwise similarities/dissimilarities
    - Sorting task: For example, define similarity by sorting objects into groups, and objects are similar if they are in the same group.
    - Condition rank order: We define similarity by ordering objects based on their similarity to some reference point (e.g. rank politicians in order of how similar they are to the politician in office)
2. Derived similarities
    - Closer to what we've done before, where we do some math based on attributes of objects to calculate a distance metric.

I think I've got that accurate to description in lecture, although it seems like all the direct similarities require a distance metric anyway?

## MDS Data

For MDS, generally, we don't have sample size requirements.
We could do MDS on a single stimulus if desired.

Note that in general, we can use either similarities or dissimilarities.
However, it is very important to keep track of what we're using.  
We also need to have a set of assumptions when we are implementing these methods in software.
The convention SPSS uses (and by extension, what we'll use) is that **all measures are dissimilarities**.

### MDS Outputs

The solution for an MDS is a set of points $X_1, X_2, \ldots, X_p$ in $r$-dimensional space.
Given the goal of visualizing the data, $r$ is typically 2 or 3.

## Metric MDS

This is the first of several approaches to MDS we'll discuss.
Here, data is in a square matrix, $\Delta$, a dissimilarity matrix.
Each matrix is assumed to have 0s on the diagonal, and be symmetric, since each $\delta_{ij}$ is representing dissimilarity between $i$ and $j$.
Obviously, a stimulus should not be dissimilar to itself, and a dissimilarity relation should be symmetric.

Now, in our final solution, the distances should preserve the dissimilarity we had to begin with.
It will attempt to make the distances as close in value to the dissimilarity as possible, but will necessarily need to approximate.
The distances are given as $d_{ij}$.
Then we can take all pairs of $d_{ij}$ and $\delta_{ij}$, and compute a correlation (or squared correlation) to see how close the solution is to the original data.  
Note that the correlation should always be positive; a negative correlation is usually an indication that we had similarity dataas inputs, I think.

So we have $\delta_{ij}$ = "perceived" dissimilarity between stimuli $i$ and $j$,  
$X_{im} = $ position on $m^\text{th}$ dimension for stimulus $i$, with $m \in [1, r]$,  
$d_{ij} = \sqrt{\sum\limits_{m=1}^r (X_{im} - X_{jm})^2}$, Euclidean distance between $i$ and $j$.

### Metric MDS Method

Note, "metric" MDS is also called "classical" MDS.

The algorithm is:
1. Take matrix $A$ where $a_{ij} = -\delta_{ij}$
and... a couple more steps. We skipped over it pretty quickly.

### Metric example

Classic example is to consider distances between cities on a map.
In theory, this should have a near-perfect 2D solution as an MDS, since the dissimilarities are already 2D (or, if including elevation, 3D) distances.
In fact, it should be able to give us a 2D representation that matches what we'd see on a map, although it could show up rotated/flipped relative to a map.  
Actually, kind of depends on shape of the original map.
First dimension is always the "most important," i.e. the dimension that maximizes distances, I think.
So, for US cities, since most spread happens east-west, we have very good chance of east-west being x-axis, and north-south being y-axis, as on a map.
For, say, Chile, we'd probably have north-south on the x-axis, since most spread of cities is north to south.

## Nonmetric MDS

In this case, we have ordinal data, say survey data that orders a bunch of items, but doesn't give an "amount" of difference from 1 to 2 or 2 to 3.
This as opposed to interval data, where the difference between a 1 and a 3 is the same as between a 5 and a 7, for example.

So, Nonmetric is meant for ordinal data, it seems.  
In general, if we get a poor correlation when using our data as interval, we can go back and ask whether we are confident the data is truly interval, and not better treated as ordinal.
For example, if we have a bunch of likert-style survey responses, we might consider switching to ordinal.
Often, ordinal, nonmetric MDS will give us a solution with higher correlation than interval, since we no longer have strict distances to match.

### Nonmetric Method

First major difference is that we need some kind of function that converts our dissimilarities to distances while preserving ordinal property.
This function is not necessarily linear, but must be monotone, so that we preserve the order.
Then we use the function as a transform from ordinal to some sort of interval representation, and we can apply the metric MDS method to our transformed data.

Basically, we will do this iteratively by trying a function, seeing what we get for distances, learning something from those distances, and going back to improve the choice of function.
Sounds like convergence is not guaranteed, though.  
For the iterative approach, we choose some metric to minimize.
For nonmetric MDS, we typically use Kruskal's STRESS index, $S = \left[ \frac{\sum\limits_{i \ne j}^n (\delta_{ij} - d_{ij})}{\sum\limits_{i \ne j}^n \delta_{ij}^2} \right]$.  
So I think we're trying to find transform $f$ that converts ordinal $\delta'_{ij}$ to interval $\delta_{ij}$, $\left(\text{i.e. } \delta_{ij} = f(\delta'_{ij})\right)$, and minimizes $S$.

Algorithm:
1. Choose number of output dimensions
2. Generate initial set of points, one for each stimulus, in output space. Typically random.
3. Compute pairwise distances $d_{ij}$ in initial configuration from (2), and graph $\delta_{ij}$ against $d_{ij}$.
4. Find least squares transform of the data, under sonstraint that it has monotonic association with original data $\delta_{ij}$. Or something. Basically, fit some kind of line to the graph of ordinal dissimilarites and interval distances from random points.
5. From that fit, we get $f$, and compute disparities (the transformed $\delta$ we get from $f(\delta')$) for each point, and compute $S$.
6. Find new configuration of points that minimizes $S$ for given set of disparities, and return to step 3.

Basically, though, we're taking random points, graphing their distances against dissimilarities, finding best-fit transform of dissimilarities to get them to match distances, do the transform, and then use those outputs to choose new points that minimize stress.

Anyway, we've got some numbers indicating when to terminate iteration.
A 20% stress is bad, a 2.5% stress is pretty good, or something, and some other numbers in between were given.

### Metric vs. Non-metric Scaling

Note that while it can seem like we're losing something by going from metric to non-metric, it can also be useful to realize that if we have some data that is "only" ordinal, we actually have a much better chance to represent it in fewer dimensions by virtue of non-metric scaling having lower bar to clear to be "good."  
So, in some sense, we generally have an easier time satisfactorially representing ordinal data in fewer dimensions than we do with interval kinds of data.

## Interpreting Dimensions

In theory, each dimension should tell us about some feature of the data.
Interpreting the dimensions, then, comes down to trying to approximately determine which feature is represented by each dimension.
Note that the solution we get could be a rotation of the best answer, so its dimensions may not align perfectly with the features.
e.g. with the morse code example, first dimension was approximately indicating the number of symbols in the letter, but was not perfectly separated in that way.

It's also possible a given dimension will do a sort of "clustering," in the sense that most items will be in a similar range of values, but a few will be "pulled away" from the group, indicating something about them stands out from most others.

## Visualizing MDS

### Visualizing Transform for Non-Metric Scaling
Recall, when doing non-metric, we use an algo to find a transform of non-metric data into some kind of metric, so we can then apply metric MDS.
One way to visualize this is a transform scatterplot.
We take "observations" as the x-axis, which are the observed dissimilarities (or similarities), and the "disparities" as the y-axis, which are the transformed dissimilarities.
We're basically just graphing the transform, then.
Recall, these are always monotone transforms, since we need to preserve order.

Now, if this transform graph is nearly linear, that suggests that our data actually would work well with a metric representation.
The transform doesn't need to change much to get a metric representation.

On the other hand, usually we'll see some nonlinear transform graph.
In this case, we're seeing how the data had to be "stretched" or "squashed" to get a good metric representation.
In morse code example, we had very little change in disparities for observations > 30.
This suggests that beyond 30 observed confusions between letters, the "amount" of similarity measure isn't so useful to distinguish the letters.

We can also graph linear fit, showing disparities on x-axis, and distances on y-axis.
In this case, we have points for all pairs.
What we'll see is a bunch of vertical spread lines, sort of.
Ok, this seems kinda fucked, I swear there's more points on here than there were in the transform one...
Best I can figure is, there were some overlapped points?
Fucking hell.
Anyway, this one should have a clear linear relationship, and the correlation we get there is our measure of how good the MDS solution is, I think.

Finally, we can plot observations on x-axis, and distances on y-axis.

### ALSCAL Algorithm

While the basic MDS algorithm has you minimizing stress, it's computationally very hard to actually solve that, apparently.
Instead, ALSCAL uses S-Stress for minimization target.
S-Stress is highly related to stress, and a low S-Stress is almost always going to correspond to low stress.
However, you should still be reporting the stress, not the S-stress.

## Potential Issues

1. Lack of convergence: We always have concern that the algorithm may not converge at all.
2. Degenerate solution(s): We can also have the algorithm give us a solution that doesn't really transform our observations into a disparity metric.
    Instead, we might get a step function, that just puts all observations on one of two (or however many) levels.
    In that case, we are probably better off using a clustering approach instead of an MDS or similar dimension reduction.
3. Local minima: As is typical for iterative algorithms, we are potentially going to find a locally-good solution, instead of the global minimum/maximum.

## Vector Projections

Use of projections in MDS is... something to do with taking vectors based on our properties of stimuli, and projecting them into the MDS space from our solution.
e.g. with morse codes, we took similarities, and got a 4D solution.  
Then we can identify some potential properties or features of the morse code letters, like number of symbols, number of dots, number of dashes, dot-dash difference, or number of "runs" in the code.
We can take these, calculate their values for each letter, and then try to regress the letters onto the solution.
However that works.

Oh, so each individual property gets regressed onto the solution.
Not quite sure how that works, when we've got a bunch of points, and now we take a single property of each letter, which corresponds to one point in the solution, which has 4 dimensions... I don't fucking get it. I guess that's what the rest of the lecture is...

But like, either we're "regressing" a single variable "down" onto 4D space, which is nonsensical, or we're "regressing" a single variable for each letter, onto a bunch of **pairwise** differences... which also seems mismatched. Ohhhh.... guess it's a multiple regression. Each of the 4 dimensions from solution are predictors, and the property is the outcome variable.
I just have zero geometric intuition for multiple regression, I guess. Dumbass.

Anyway, steps to the process:
1. Calculate all properties/features identified
2. Standardize the props, so each property has a distribution with mean 0 and std. dev. of 1.
3. Merge our new properties as columns in the original data (not really an algorithmic step, but practically something useful for software work).
4. For each property, do a multiple regression against the dimensions from the MDS solution.

### Measuring Fit

We define a measure $c = \frac{R}{\sqrt{b_1^2 + b_2^2 + \ldots + b_m^2}}$, where $b$ are the individual raw (non-standardized) coefficients, which tells us how long the vector of our variable is in the $m$-dimensional space defined by the MDS solution.
And then I think the coefficients themselves are the direction in that space?
Oh, or something about we multiply any given coefficient by $c$ to get the length of that component of the vector of the property in the space of the solution on the corresponding axis. Or some bullshit.
Also, there's something about standardized vs. unstandardized coefficients, however the fuck those came about. Maybe standardized is just what we get from that multiplication? No, no it's clearly not. 